# ADM-HW4: GROUP #10

#### Xavier Del Giudice


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
|  Valeria Avino| 1905974 |   avino.1905974@studenti.uniroma1.it                                      |
|  |  |                       |

## Import and Install required packages

In [2]:
%pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import re
import time

# 1. Recommendation System with LSH

## 1.1 Data Preparation

Reading the dataset and checking (if any) missing data.

In [172]:
#reading from our dataset
genome_scores = pd.read_csv('MovieLens/genome_scores.csv')
genome_tags = pd.read_csv('MovieLens/genome_tags.csv')
link = pd.read_csv('MovieLens/link.csv')
movie = pd.read_csv('MovieLens/movie.csv')
rating = pd.read_csv('MovieLens/rating.csv')
tags = pd.read_csv('MovieLens/tag.csv')

In [12]:
genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


In [13]:
#checking missing values
missing_values = genome_scores.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [14]:
genome_scores.duplicated().sum()

0

scores dataset has no missing values or duplicated rows

In [15]:
genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [16]:
missing_values = genome_tags.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [17]:
genome_tags.duplicated().sum()

0

In [18]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [19]:
missing_values = link.isnull().sum()
missing_values[missing_values > 0]

tmdbId    252
dtype: int64

In [20]:
link.duplicated().sum()

0

In [21]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [22]:
movie.duplicated().sum()

0

In [23]:
missing_values = movie.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [24]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [25]:
missing_values = rating.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [26]:
rating.duplicated().sum()

0

In [173]:
rating = rating.rename(columns={'timestamp':'rating_timestamp'})

In [28]:
highest_rated = rating.loc[rating['rating'].idxmax(), 'movieId']
lowest_rated = rating.loc[rating['rating'].idxmin(), 'movieId']
#reading titles from movie dataset
print(f'The highest rated movie is {movie.loc[movie['movieId'] == highest_rated, 'title' ].values[0]} ')
print(f'The lowest rated movie is {movie.loc[movie['movieId'] == lowest_rated, 'title'].values[0]} ')

The highest rated movie is Lord of the Rings: The Fellowship of the Ring, The (2001) 
The lowest rated movie is Nemesis 2: Nebula (1995) 


In [29]:
# splitting genres column to do store it as a list
movie['genres'] = movie['genres'].str.split('|')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [30]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userId     465564 non-null  int64 
 1   movieId    465564 non-null  int64 
 2   tag        465548 non-null  object
 3   timestamp  465564 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.2+ MB


In [31]:
missing_values = tags.isnull().sum()
missing_values[missing_values > 0]
#we could drop this rows or just keep them to check which user did not give a tag for which movies

tag    16
dtype: int64

In [32]:
tags.duplicated().sum()

0

In [174]:
tags = tags.rename(columns={'timestamp':'tag_timestamp'})

### Merging the datasets on common columns

In [34]:
#merging tags and genom_tags on 'tag' column
merged_tags = pd.merge(tags, genome_tags, on ='tag', how = 'left')
merged_tags.head()

,userId,movieId,tag,tag_timestamp,tagId
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0
2,65,353,dark hero,2013-05-10 01:41:19,288.0
3,65,521,noir thriller,2013-05-10 01:39:43,712.0
4,65,592,dark hero,2013-05-10 01:41:18,288.0


In [35]:
#tag ids are converted into floats because od Nan values
#let's convert them back to nullable ints
merged_tags['tagId'] = merged_tags['tagId'].astype('int64',errors='ignore')

In [36]:
merged_tags.head()

,userId,movieId,tag,tag_timestamp,tagId
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0
2,65,353,dark hero,2013-05-10 01:41:19,288.0
3,65,521,noir thriller,2013-05-10 01:39:43,712.0
4,65,592,dark hero,2013-05-10 01:41:18,288.0


In [37]:
# joining the genome_scores to add the tag relevance
merged_tag = pd.merge(merged_tags, genome_scores, on = ['movieId', 'tagId'], how='left')

In [38]:
merged_tag.head()

,userId,movieId,tag,tag_timestamp,tagId,relevance
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0,0.94950
2,65,353,dark hero,2013-05-10 01:41:19,288.0,0.99800
3,65,521,noir thriller,2013-05-10 01:39:43,712.0,0.98900
4,65,592,dark hero,2013-05-10 01:41:18,288.0,0.99775


In [39]:
df = pd.merge(rating, merged_tag, on = ['userId', 'movieId'], how = 'left')
df.head()

,userId,movieId,rating,rating_timestamp,tag,tag_timestamp,tagId,relevance
0,1,2,3.5,2005-04-02 23:53:47,NaN,NaN,NaN,NaN
1,1,29,3.5,2005-04-02 23:31:16,NaN,NaN,NaN,NaN
2,1,32,3.5,2005-04-02 23:33:39,NaN,NaN,NaN,NaN
3,1,47,3.5,2005-04-02 23:32:07,NaN,NaN,NaN,NaN
4,1,50,3.5,2005-04-02 23:29:40,NaN,NaN,NaN,NaN


In [40]:
df = pd.merge( df, movie, on ='movieId', how='left')

In [41]:
df.head()

,userId,movieId,rating,rating_timestamp,tag,tag_timestamp,tagId,relevance,title,genres
0,1,2,3.5,2005-04-02 23:53:47,NaN,NaN,NaN,NaN,Jumanji (1995),"[Adventure, Children, Fantasy]"
1,1,29,3.5,2005-04-02 23:31:16,NaN,NaN,NaN,NaN,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]"
2,1,32,3.5,2005-04-02 23:33:39,NaN,NaN,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]"
3,1,47,3.5,2005-04-02 23:32:07,NaN,NaN,NaN,NaN,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,3.5,2005-04-02 23:29:40,NaN,NaN,NaN,NaN,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"


In [42]:
df['userId'] = df['userId'].astype('int64')
df['tagId'] = df['tagId'].astype('int64', errors= 'ignore')
df.head()

,userId,movieId,rating,rating_timestamp,tag,tag_timestamp,tagId,relevance,title,genres
0,1,2,3.5,2005-04-02 23:53:47,NaN,NaN,NaN,NaN,Jumanji (1995),"[Adventure, Children, Fantasy]"
1,1,29,3.5,2005-04-02 23:31:16,NaN,NaN,NaN,NaN,"City of Lost Children, The (Cité des enfants p...","[Adventure, Drama, Fantasy, Mystery, Sci-Fi]"
2,1,32,3.5,2005-04-02 23:33:39,NaN,NaN,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),"[Mystery, Sci-Fi, Thriller]"
3,1,47,3.5,2005-04-02 23:32:07,NaN,NaN,NaN,NaN,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,3.5,2005-04-02 23:29:40,NaN,NaN,NaN,NaN,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"


In [43]:
print(df.rating.isnull().sum())

0


In [3]:
df = pd.read_csv('complete_dataset.csv')

In [ ]:
# Final dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20265625 entries, 0 to 20265624
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   userId            int64  
 1   movieId           int64  
 2   rating            float64
 3   rating_timestamp  object 
 4   tag               object 
 5   tag_timestamp     object 
 6   tagId             float64
 7   relevance         float64
 8   title             object 
 9   genres            object 
dtypes: float64(3), int64(2), object(5)
memory usage: 1.5+ GB


## 1.2 Minhash Signatures
*"Using the __`userId`__ and __`movieId`__ columns, implement your own **MinHash** function.
This function will hash each user's watched movie list, creating a representation that allows for quick comparisons of user similarities."*

In order to create the signature matrix for our lists of seen movies we have implemented the following functions in the `minhashing.py` file:

+ The function `char_vec` creates a **characteristic vector** for a user's list of seen movies. It iterates over all possible movie IDs (`all_movies`) and assigns a `1` if a movie is in the user's seen list (`seen_movies`), and `0` otherwise. The result is a binary vector indicating the presence or absence of each movie for that user. Finally it converts the result into a memory-efficient sparse matrix using compressed sparse row (CSR) format.

+ The `minhash` function generates a **MinHash signature** for a given characteristic vector. This signature is a compact representation used in Locality Sensitive Hashing (LSH) to approximate Jaccard similarity efficiently. The function applies a set of **hash functions** to the indices of the vector, updating the signature values **only** for indices where the vector value is `1`. The result is a vector where each entry corresponds to the *minimum hashed index* encountered for each hash function.


In [176]:
import minhashing
from minhashing import char_vec, minhash

In [177]:
# list of all movie ids
all_movies = df["movieId"].unique()

# creating a filtered dataset selecting from df the 'userId' column 
# and aggregating on it 'movieIds'  to create a list of user's seen/rated movies 

user_seen_movies = df.groupby("userId")["movieId"].agg(set).reset_index().rename(columns={"movieId": "seen_movies"})
print(f"We  will work with {user_seen_movies.shape[0]} classes, since that is the number of users")

We  will work with 138493 classes, since that is the number of users


In [178]:
user_seen_movies.head()

,userId,seen_movies
0,1,"{2, 3081, 4105, 1036, 5146, 541, 29, 32, 4128,..."
1,2,"{3, 260, 2948, 2951, 1544, 1673, 266, 908, 245..."
2,3,"{512, 1, 3072, 2053, 2054, 2567, 1544, 2571, 1..."
3,4,"{6, 519, 520, 10, 19, 531, 32, 420, 165, 548, ..."
4,5,"{2, 515, 260, 1028, 648, 11, 140, 141, 780, 10..."


+ Sort the `all_movies` list so that, for $ m = \text{len(seen\_movies)} $, the characteristic vector of user $j$, denoted as $\text{char\_vec}(\text{user}_j) = [x_1^j, x_2^j, \dots, x_m^j]$, is defined as follows:  
$$
x_i^j = 
\begin{cases} 
1 & \text{if user } j \text{ has seen (rated) the } i\text{-th movie in the list}, \\
0 & \text{otherwise}.
\end{cases} \quad
\forall i\quad\in\{1,...,m\}$$

In [179]:
all_movies = sorted(all_movies)
print(f"Movie ids range from {all_movies[0]} to {all_movies[-1]}: we will work with {all_movies[-1]-all_movies[0]+1} shingles, each of which is one distinct movieId.")

Movie ids range from 1 to 131262: we will work with 131262 shingles, each of which is one distinct movieId.


* Compute the **characteristic vector** for each class, applying the `char_vec` function on `seen_movies` column.

In [181]:
user_seen_movies['char_vec'] = user_seen_movies['seen_movies'].apply(lambda x: char_vec(x, all_movies))

In [182]:
print(user_seen_movies.iloc[0]['char_vec'].toarray())

[[0 1 0 ... 0 0 0]]


* Define multiple independent **hash functions** to permute shingles:

    For a given $k = \text{number of hash functions} $ we will compute in `minhash` function
    $$ h(x) = ( a_ix+b_i ) \text{ mod  } p $$
    where $a = [ a_1, ..., a_k]$ and $b= [ b_1, ..., b_k]$ are random coefficient vectors in $\mathbb{N} ^k$, such that $a_i>0 \quad\forall i$ and $p$ is a **prime** larger then our number of shingles.

    This definition ensures that the hash functions are **independent**, being a unique permutation of the shingles, and **uniformly** map the shingles minimizing collisions,  thanks to the modulo division with a prime.

    To analyze the performance of our `minhash` implementation, we will experiment with various values of $k$.

In [184]:
import random

num_shingles = len(all_movies)

# adjust the signature vectors' lenght ( e.g. start with k = 3 )
k = 3

# Set a prime number greater than num_shingles
prime = 131273

# Generate random coefficients
random.seed(42)  # For reproducibility
a = np.random.randint(1, prime, size = k, dtype=np.int64)
b = np.random.randint(0, prime, size = k, dtype=np.int64)


* Now apply to each characteristic vector the `minhash` function to generate the associated **MinHash signature vectors**.

In [185]:
user_seen_movies['minhash'] = user_seen_movies['char_vec'].apply(
    lambda x: minhash(x, k, a, b, prime)
)

In [186]:
user_seen_movies['minhash'].head()

0       [881, 454, 386]
1      [881, 252, 1322]
2      [114, 397, 1607]
3     [114, 5335, 1349]
4    [2661, 4452, 1634]
Name: minhash, dtype: object

Deploying the MinHash property that *"the similarity of the signatures is the fraction of the min-hash functions (rows) in which they agree"* we can approximate the Jaccard similarity between two user's seen movie list $x_1$ and $x_2$ as $$ sim (x_1, x_2) =  \mathbb{P}[ h(x_1) = h(x_2)]$$

We can check if we have implemented a reliable proxy for the given $k = 3$ applying `jaccard_sim` and `sign_agreement` functions implemented in `tools.py`. The former evaluates the jaccard similarity between to sets, while the latter returns the frequency of agreeing components of the respective signature vectors.

In [188]:
from tools import jaccard_sim
for i in range(1,5):
    sim0i = jaccard_sim(user_seen_movies.iloc[0]['seen_movies'], user_seen_movies.iloc[i]['seen_movies'])
    print(f"sim(0,{i}) = {sim0i}")

sim(0,1) = 0.039647577092511016
sim(0,2) = 0.13125
sim(0,3) = 0.015
sim(0,4) = 0.06167400881057269


In [198]:
from tools import sign_agreement
for i in range(1,5):
    print(sign_agreement(user_seen_movies.iloc[0]['minhash'],user_seen_movies.iloc[i]['minhash']))

0.3333333333333333
0.0
0.0
0.0


These test values are enough to state that selecting a small $k$ (e.g. $k = 3$) does not provide an accurate approximation of similarity. We need to adjust our parameter accordingly.

In [206]:
#increasing k
k = 25
random.seed(42)  # For reproducibility
a = np.random.randint(1, prime, size = k, dtype=np.int64)
b = np.random.randint(0, prime, size = k, dtype=np.int64)
user_seen_movies['minhash_25'] = user_seen_movies['char_vec'].apply(lambda x: minhash(x, k, a, b, prime))

In [201]:
from tools import sign_agreement
for i in range(1,5):
    print(sign_agreement(user_seen_movies.iloc[0]['minhash_25'],user_seen_movies.iloc[i]['minhash_25']))

0.0
0.12
0.0
0.04


In [205]:
k = 50
random.seed(42)  # For reproducibility
a = np.random.randint(1, prime, size = k, dtype=np.int64)
b = np.random.randint(0, prime, size = k, dtype=np.int64)
user_seen_movies['minhash_50'] = user_seen_movies['char_vec'].apply(lambda x: minhash(x, k, a, b, prime))

In [204]:
for i in range(1,5):
    print(sign_agreement(user_seen_movies.iloc[0]['minhash_50'],user_seen_movies.iloc[i]['minhash_50']))

0.04
0.1
0.0
0.04


**Increasing $k$** improves the reliability and accuracy of the MinHash-based similarity approximation because it reduces randomness and provides more robust estimates by leveraging multiple independent hash functions. 

At the same time we can expect practical **limitations** to making $k$ too large, as:
- Memory usage: the signature matrix sizes increases with $k$;
- Accuracy loss: the improvement in the similarity estimate becomes less significant as more hash functions are added:

While increasing $k$ can improve the accuracy of MinHash, there is a **trade-off** with computational cost, memory usage, and diminishing improvements in similarity approximation. We found in $k = 50$ a good candidate solution, but we can keep working with diffeent values and adjust other parameters too.

## 1.3 Locality-Sensitive Hashing (LSH)

### 1. Bucket Creation

In [215]:
# creating the mihash signature matrix for k = 25 stacking MinHash vectors as columns

signature_matrix = np.array(user_seen_movies['minhash_25'].tolist()).T
print("Signature Matrix Shape:", signature_matrix.shape)


Signature Matrix Shape: (25, 138493)


To build the buckets we will use the `lsh` function from `Locality_sensitive_Hasing.py` file, which returns a list of dictionaries to map band signatures to lists of items (columns, which resemble users) that share the same signature in that band. It is structured as follows:

- <u>Banding:</u>

    The signature matrix is divided into *bands*, with each band containing a subset of the rows from the signature matrix.

- <u>Hashing Mini-Signatures:</u>

    Each band is hashed into buckets. The hash function in this case is a tuple of the band rows, which represents the signature of an item for that band.
    If two items (columns) share the same band signature, they are placed in the same bucket for that band.

- <u>Buckets and Similarity</u>:

   Items that share buckets in multiple bands are considered similar and have a high probability of being similar based on their Jaccard similarity.

The resulting `buckets` list contains one dictionary per band, where each dictionary maps band signatures to a list of item indices with that signature.

In [216]:
import Locality_Sensitive_Hashing
from Locality_Sensitive_Hashing import lsh

band_size = 5  # Number of rows per band (e.g.  k/bandisize = 25/5 bands)
buckets = lsh(signature_matrix, band_size)

# Print example of bucket structure
for i, band in enumerate(buckets[:5]):
    print(f"Band {i + 1}:")
    
    for j, (signature, items) in enumerate(band.items()):
        if j >= 3:
            break  # Stop after printing a few buckets
        print(f"  Signature {signature}: Items {items}")

Band 1:
  Signature (297, 3150, 462, 938, 2442): Items [0]
  Signature (5859, 8149, 1586, 334, 0): Items [1]
  Signature (605, 381, 389, 454, 1784): Items [2]
Band 2:
  Signature (147, 4, 1555, 51, 584): Items [0]
  Signature (1292, 1927, 87, 1258, 421): Items [1]
  Signature (147, 1803, 337, 113, 72): Items [2, 2512, 48740, 55758, 58388, 106387, 115574, 124672]
Band 3:
  Signature (288, 1226, 930, 2351, 131): Items [0]
  Signature (1745, 645, 258, 985, 768): Items [1]
  Signature (97, 346, 450, 147, 1217): Items [2]
Band 4:
  Signature (366, 42, 700, 821, 669): Items [0]
  Signature (5235, 1788, 698, 221, 11310): Items [1]
  Signature (486, 1163, 328, 179, 514): Items [2]
Band 5:
  Signature (473, 360, 2812, 759, 1129): Items [0]
  Signature (2029, 2555, 3984, 100, 4345): Items [1]
  Signature (328, 445, 2017, 19, 643): Items [2]


### 2. Query
*"For a given user, identify the two most similar users based on their bucket placement. If a user doesn’t have any similar users in their bucket, adjust the parameters until similar users are found."*

Our `top2_query` implementation works as follows:

- The function computes the "band signature" for the target user across all bands.
- It then compares this signature with others in the same bucket to count the occurrences of similar users.
- Only users with a similarity count exceeding the threshold are considered.
- The users are sorted based on their similarity count, and the top two similar users are returned.
- If no users meet the threshold, a message suggesting parameter adjustment is returned.

In [223]:
from Locality_Sensitive_Hashing import top2_query

thresh = 1
result =  top2_query( user = 2512, signature_matrix = signature_matrix, band_size = 5, buckets = buckets, threshold=thresh)
print(f'Most similar users that appear in more then {thresh} buckets:\n {result}')

Most similar users that appear in more then 1 buckets:
 [109130]


In [224]:
#First adjustment: lower threshold to 0 to find results
# (results are returned if users appear in the same bucket for at least one band)

result =  top2_query( user = 2512, signature_matrix = signature_matrix, band_size = 5, buckets = buckets, threshold=0)
print(f'Most similar users that appear in more then {thresh} buckets:\n {result}')


Most similar users that appear in more then 1 buckets:
 [109130, 3938]


If $k$ is too small (even $k =  25$), searching for matching bands essentially becomes equivalent to searching for matching signature vectors in their entirety. To ensure results to our query, even with a threshold greater than 0, we should increase the value of $k$.

In [225]:
# Increasing k from 25 to 50, lowering bandsize to 2, increasing threshold to 2 

signature_matrix_50 = np.array(user_seen_movies['minhash_50'].tolist()).T
buckets_50 = lsh (signature_matrix=signature_matrix_50, band_size=2)
print("Signature Matrix Shape:", signature_matrix_50.shape)

new_result =  top2_query( 2512, signature_matrix_50, 2, buckets_50, 2)
print(f'Most similar users that that appear in more then 2 buckets :\n {new_result}')


Signature Matrix Shape: (50, 138493)
Most similar users that that appear in more then 2 buckets :
 [114896, 53011]


### 3-4. Movie Recommendation Logic and Final Recommendation
*"Movie Recommendation Logic:*
+ *If both similar users have rated a movie, recommend this movie based on the **average rating**.*
+ *If there are no commonly rated movies, recommend the top-rated movies of the most similar user.*

"Final Recommendation: Provide at most five movies to the user."

In [226]:
from Locality_Sensitive_Hashing import recommend_movie
recommend_movie(user_seen_movies, result, df)

('Top 5 recommended movies based on common ratings:',
    movieId  avg_rating                             title
 0      110         5.0                 Braveheart (1995)
 1      296         5.0               Pulp Fiction (1994)
 2      111         5.0                Taxi Driver (1976)
 3      593         5.0  Silence of the Lambs, The (1991)
 4      356         4.5               Forrest Gump (1994))

In [227]:
from Locality_Sensitive_Hashing import recommend_movie
recommend_movie(user_seen_movies, new_result, df)

('Top 5 recommended movies based on common ratings:',
    movieId  avg_rating                             title
 0       50         5.0        Usual Suspects, The (1995)
 1       39         4.5                   Clueless (1995)
 2      296         4.5               Pulp Fiction (1994)
 3      111         4.5                Taxi Driver (1976)
 4      593         4.0  Silence of the Lambs, The (1991))

The choice of the parametres results in different movies suggestion, which are affected by the LSH similarity between the minhash signature, that highly depends on $k$, *band_size* and *threshold* values.

# 2. Grouping Movies Together!

## 2.1 Feature Engineering

## 2.2 Choose your features (variables)!

## 2.3 Clustering

## 2.4 Best Algorithm

# 3. Bonus Question

# 4. Algorithmic Question